In [1]:
%matplotlib inline
import brian2 as b2
import matplotlib.pyplot as plt
import numpy as np
from neurodynex.leaky_integrate_and_fire import LIF
import neurodynex.exponential_integrate_fire.exp_IF as exp_IF
from neurodynex. tools import input_factory, plot_tools
from neurodynex.adex_model import AdEx


In [34]:
"""
Implements the dynamics of the exponential Integrate-and-fire model

Args:
    tau (Quantity): Membrane time constant
    R (Quantity): Membrane resistance
    v_rest (Quantity): Resting potential
    v_reset (Quantity): Reset value (vm after spike)
    v_rheobase (Quantity): Rheobase threshold
    v_spike (Quantity) : voltage threshold for the spike condition
    delta_T (Quantity): Sharpness of the exponential term
    I_stim (TimedArray): Input current
    simulation_time (Quantity): Duration for which the model is simulated
"""
v_rest= -60*b2.mV #EL
v_reset= -50*b2.mV #Vreset
conductance_L = 10*b2.nsiemens #GL
membrane_capacitance = 100*b2.pF #CM
firing_threshold = -40* b2.mV #Vth
deltaVth = 2*b2.mV

a = 0.03/(b2.ms)
b = -2*(b2.mV/b2.pA)
k = 0.7*(b2.mV/b2.pA)
d = 100*b2.pA 

tau_w = 200*b2.ms
v_max = 35*b2.mV

simulation_time = 1500

#current = 200.01 * b2.pamp
current = 200
input_current = input_factory.get_step_current(
t_start=500, t_end=1000, unit_time=b2.ms,
amplitude=current*b2.pA) 

# differential equation of Leaky Integrate-and-Fire model
eqs = """
dv/dt = ((k/membrane_capacitance)*(v - v_rest)*(v-firing_threshold) - (u/membrane_capacitance) + (input_current(t,i)/membrane_capacitance)) : volt
du/dt = a*(b*(v - v_rest) - u) : amp
"""
neuron = b2.NeuronGroup(1, model=eqs, reset="v=v_reset; u=u+b", threshold="v>v_max", method="euler")
neuron.v = v_rest
neuron.u = 0



# monitoring membrane potential of neuron and injecting current
state_monitor = b2.StateMonitor(neuron, ["v", "u"], record=True)
spike_monitor = b2.SpikeMonitor(neuron)
# run the simulation
b2.run(simulation_time*b2.ms)

print("nr of spikes: {}".format(spike_monitor.count[0]))
print("spikes times: {}".format(spike_monitor.t[:]))

plt.figure(1)
c = input_current(state_monitor.t, 0)
plt.plot(state_monitor.t/b2.ms, c/b2.pA )
plt.xlabel('tempo (ms)')
plt.ylabel('Iapp (pA)')
plt.grid()
plt.figure(2)
#axis_v = plt.subplot(312)

plt.plot(state_monitor.t/b2.ms, state_monitor.v[0]/b2.mV)
plt.xlabel('tempo (ms)')
plt.ylabel('V (mV)')
plt.grid()
#plt.legend(["vm", "firing threshold"], fontsize=12)
plt.figure(3)

plt.plot(state_monitor.t/b2.ms, state_monitor.w[0]/b2.pA)
plt.grid()
plt.xlabel('tempo (ms)')
plt.ylabel('w (pA)')

plt.show()


BrianObjectException: Original error and traceback:
Traceback (most recent call last):
  File "D:\Programas\Miniconda2\envs\bmnn\lib\site-packages\brian2\core\network.py", line 841, in before_run
    obj.before_run(run_namespace)
  File "D:\Programas\Miniconda2\envs\bmnn\lib\site-packages\brian2\groups\neurongroup.py", line 811, in before_run
    self.equations.check_units(self, run_namespace=run_namespace)
  File "D:\Programas\Miniconda2\envs\bmnn\lib\site-packages\brian2\equations\equations.py", line 963, in check_units
    *ex.dims)
DimensionMismatchError: Inconsistent units in differential equation defining variable u:
Expression "b * (v - v_rest) - u" uses inconsistent units ("b * (v - v_rest)" has unit metre ** 4 * kilogram ** 2 * second ** -6 * amp ** -3; "u" has unit amp)

Error encountered with object named "neurongroup_1".
Object was created here (most recent call only, full details in debug log):
  File "<ipython-input-34-b78efec96c32>", line 43, in <module>
    neuron = b2.

In [ ]:
v_rest= -75*b2.mV #EL
v_reset= -80*b2.mV #Vreset
conductance_L = 10*b2.nsiemens #GL
membrane_capacitance = 100*b2.pF #CM
firing_threshold = -50 * b2.mV #Vth
deltaVth = 2*b2.mV

a = 2*b2.nsiemens
b = 0.02*b2.nA

tau_w = 200*b2.ms
v_max = 20*b2.mV

#current = 200.01 * b2.pamp
input_current = np.arange(300, 600, 10)
# differential equation of Leaky Integrate-and-Fire model
eqs = """

dv/dt = (conductance_L/membrane_capacitance)*(v_rest - v + deltaVth*exp((v-firing_threshold)/deltaVth))
        - (w/membrane_capacitance)
        + (input_current/membrane_capacitance) : volt
dw/dt = (a/tau_w) * (v - v_rest) - (w/tau_w) : ampere
input_current : amp
"""
neuron = b2.NeuronGroup(len(current), model=eqs, reset="v=v_reset; w=w+b", threshold="v>v_max", method="euler")
neuron.v = v_rest
neuron.w = 0*b2.pA
neuron.input_current = input_current*b2.pA

# monitoring membrane potential of neuron and injecting current
state_monitor = b2.StateMonitor(neuron, ["v","w"], record=True)
spike_monitor = b2.SpikeMonitor(neuron)
# run the simulation
b2.run(simulation_time*b2.second)

#plot
# input_current = [None]*len(current)
# for i in range(len(current)):
#     input_current[i] = input_factory.get_step_current(t_start=0, t_end=simulation_time, unit_time=b2.second,
#     amplitude= current[i]*b2.nA) 
#     plot_tools.plot_voltage_and_current_traces(
#     state_monitor, input_current[i], title="min input", firing_threshold=firing_threshold)

plt.figure(2)
print("nr of spikes: {}".format(spike_monitor.count))
print("spikes times: {}".format(spike_monitor.t[:]))

print("spikes train: {}".format(spike_monitor.spike_trains()[0][0]))

first_inter = [None]*len(input_current)
stationary_inter = [None]*len(input_current)
for i in range(len(input_current)):
    first_inter[i] = 1/(float(spike_monitor.spike_trains()[i][2]) - float(spike_monitor.spike_trains()[i][1]))
    stationary_inter[i] = 1/(float(spike_monitor.spike_trains()[i][-2]) - float(spike_monitor.spike_trains()[i][-3]))
    
print(first_inter)
print(stationary_inter)
plt.plot(neuron.input_current/b2.pA, spike_monitor.count/simulation_time, '*', label='count')
plt.plot(neuron.input_current/b2.pA, first_inter/b2.second, 's', label='first_inter')
plt.plot(neuron.input_current/b2.pA, stationary_inter/b2.second, 'o', label='stationary_inter')
plt.xlabel('I (nA)')
plt.ylabel('Firing rate (sp/s)')
plt.legend()
plt.show()#%%
